In this Jupyter Notebook we will code from scratch a CNN

In [16]:
### Imports ###

import numpy as np
from scipy import signal

np.random.seed(0)  # For reproductibility

I - Mother class

In [17]:
class layer :

    def __init__(self) :

        self.input = None
        self.output = None

    def forward(self, input) :
        pass

    def backward(self, grad, eta) :    # eta : learning rate
        pass

II - Dense layer

In [18]:
class dense_layer(layer) :

    def __init__(self, nb_inputs, nb_neurones) :

        self.weights  = np.random.standard_normal((nb_neurones, nb_inputs))
        self.biases = np.random.standard_normal((nb_neurones,1))

    def forward(self, input) :

        self.input = input

        return np.dot(self.weights, self.input) + self.biases

    def backward(self, grad, eta) :

        weights_grad = np.dot(grad, self.input.T)

        self.weights -= eta*weights_grad      # weights update
        self.biases -= eta*grad             # biases update

    
        return np.dot(self.weights.T, grad)

III - Convolutionnal layer

In [19]:
class convolutionnal_layer(layer) :

    def __init__(self, input_shape, kernel_size, depth) :


        # We first deal with the dimension of our images, kernes and features

        input_depth, input_height, input_width =  input_shape

        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)

        # We iniatlize the kernels and the biases

        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, input) :

        self.input = input
        self.output = np.copy(self.biases)

        for i in range(self.depth) :
            for j in range(self.input_depth) :

                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i,j], 'valid')

        return self.output

    def backward(self, grad, eta) :

        kernels_grad = np.zeros(self.kernels_shape)
        input_grad = np.zeros(self.input_shape)

        for i in range(self.depth) :
            for j in range(self.input_depth) :

                kernels_grad[i,j] = signal.correlate2d(self.input[j], grad[i], 'valid')
                input_grad[j] +=  signal.convolve2d(grad[i], self.kernels[i,j], 'full')

        self.kernels -= eta * kernels_grad
        self.biases -= eta * grad

        return input_grad


III - Activation layer

In [20]:
class sigmoid_activation_layer(layer) :

    def __init__(self) :

        self.activ_func = lambda x : 1/(1 + np.exp(-x))
        self.derivative = lambda x : 1/(1 + np.exp(-x))*(1 - 1/(1 + np.exp(-x)))

    def forward(self, input) :

        self.input = input

        return self.activ_func(self.input)

    def backward(self, grad, eta) :

        return np.multiply(grad, self.derivative(self.input))

IV - Reshape layer

In [21]:
class reshape_layer(layer) :

    def __init__(self, input_shape, output_shape) :

        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input) :

        return np.reshape(input, self.output_shape)

    def backward(self, grad, eta) :

        return np.reshape(grad, self.input_shape)

V - Pooling layer

In [22]:
class avg_pool_layer(layer) :

    def __init__(self, input_shape, kernel_size) :


        # We first deal with the dimension of our images, kernes and features

        input_depth, input_height, input_width =  input_shape

        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (input_depth, input_height - kernel_size + 1, input_width - kernel_size + 1)

        # We iniatlize the kernel 

        self.kernels = kernel_size**(-1)*np.ones((input_depth,kernel_size,kernel_size))

    def forward(self, input) :

        self.input = input
        self.output = np.zeros(self.output_shape)

        for i in range(self.input_depth) :

            self.output[i] += signal.correlate2d(self.input[i], self.kernels[i], 'valid')

        return self.output

    def backward(self, grad, eta) :

        input_grad = np.zeros(self.input_shape)

        for i in range(self.input_depth) :

            input_grad[i] +=  signal.convolve2d(grad[i], self.kernels[i], 'full')

        return input_grad

VI- Loss layer

In [23]:
class cross_entropy : 

    def __init__(self, y_pred, y_true) : 

        self.y_pred = np.clip(y_pred, 10e-7, 1 - 10e-7)
        self.y_true = y_true

    def compute(self) :

        error = -np.dot(np.log(self.y_pred).T,self.y_true)
        
        return float(np.average(error))
    
    def grad(self) : 

        grad = np.sum(np.divide(self.y_true,self.y_pred))


        return -grad.size**(-1)*grad